<a href="https://colab.research.google.com/github/tom5610/my-fsdl-text-recognizer-2022/blob/main/lab01/notebooks/c_lab01_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 01: Deep Neural Networks in PyTorch

Objective:
* how to write a basic neural network from scatch in PyTorch
* How the submodules of torch, like `torch.nn` and `torch.utils.data`, make writing performant neural network training and inference code easier

## Setup
Below cell will run full environment setup in colab.

In [ ]:
lab_idx = 1

if 'bootstrap' not in locals() or bootstrap.run:
    pythonpath, = !echo $PYTHONPATH
    if '.' not in pythonpath.split(':'):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow 'hot-reloading' of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False

!pwd
%ls

env: PYTHONPATH=.:/env/python
.:/env/python
/content/fsdl-text-recognizer-2022-labs/lab01
notebooks/  text_recognizer/


## Getting data and making Tensors

before we can build a model, we need data. 

Use MNIST dataset of handwritten digits.

The data used to train state-of-the-art models these days is generally too large to be stored on the disk of any single machine (to say nothing of the RAM!), so fetching data over a network is a common first step in model training.

In [ ]:
from pathlib import Path 
import requests

def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename

data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

In [ ]:
import gzip
import pickle 

def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)
    

In [ ]:
import torch 

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [ ]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


In [ ]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

In [ ]:
x_train.ndim, y_train.ndim

(2, 1)

In [ ]:
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


In [ ]:
import random 

import wandb 

import text_recognizer.metadata.mnist as metadata

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])
wandb.Image(example.reshape(*metadata.DIMS)).image

tensor(1)


## Build a DNN using only `torch.Tensor` methods and Python

### Defining the model

In [ ]:
import math

import torch
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [ ]:
weights.ndim, weights.shape

(2, torch.Size([784, 10]))

In [ ]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [ ]:
n = 3
a = torch.rand(n)
b = torch.rand(n)
print(torch.matmul(a, b))
print(a @ b)
a, b

tensor(0.4026)
tensor(0.4026)


(tensor([0.5367, 0.3610, 0.3556]), tensor([0.0799, 0.3948, 0.6106]))

In [ ]:
0.5367 * 0.0799 + 0.361 * 0.3948 + 0.3556 * 0.6106

0.40253448999999997

In [ ]:
a = torch.rand((3, 2))
b = torch.rand((2, 2))
result = a.mm(b)

In [ ]:
a, b, result

(tensor([[0.4673, 0.3693],
         [0.5010, 0.0284],
         [0.0433, 0.7331]]), tensor([[0.1842, 0.0664],
         [0.6001, 0.8962]]), tensor([[0.3077, 0.3620],
         [0.1093, 0.0587],
         [0.4479, 0.6599]]))

In [ ]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis = 1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [ ]:
bs = 64 # batch size

xb = x_train[0: bs] # a batch of inputs
outs = model(xb)

print(outs[0], outs.shape)


tensor([-2.3889, -2.3560, -2.0397, -1.6576, -2.3886, -2.7267, -2.2953, -2.9370,
        -2.4488, -2.3492], grad_fn=<SelectBackward0>) torch.Size([64, 10])


## Defining the loss and metrics

In [ ]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    print('accuracy comparison - ', (preds == yb).float())
    return (preds == yb).float().mean()

In [ ]:
yb = y_train[0: bs]

acc = accuracy(outs, yb)

print(acc)

accuracy comparison -  tensor([0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor(0.0938)


In [ ]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [ ]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.3887, grad_fn=<NegBackward0>) tensor(2.3026)


In [ ]:
torch.log(torch.tensor(1 / 10))

tensor(-2.3026)

In [ ]:
loss = loss_func(outs, yb)

loss.backward()

In [ ]:
weights.grad, bias.grad

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([-0.0202, -0.0252,  0.0520,  0.0107, -0.0019,  0.0245,  0.0063, -0.0183,
          0.0081, -0.0360]))

## Defining and running the fitting loop

We now have all the ingredients we need to fit a neural network to data:

* data (x_train, y_train)
* a network architecture with parameters (model, weights, and bias)
* a loss_func to optimize (cross_entropy) that supports .backward computation of gradients

In [ ]:
lr = 0.5
epochs = 2

for epoch in range(epochs): # loop over the data repeatedly
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)

        # calculate the gradients with a backwards pass
        loss.backward()

        # update the parameters
        with torch.no_grad(): # we don't want to track gradients through this part!
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # achtung: PyTorch doesn't assume you're done with gradients
            #.         until you say so -- by explicitly "deleting" them. i.e. setting the gradients to 0.
            weights.grad.zero_()
            bias.grad.zero_()


In [ ]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

accuracy comparison -  tensor([0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
        1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0.])
tensor(1.2205, grad_fn=<NegBackward0>) tensor(0.7500)


In [ ]:
idx = random.randint(0, len(x_train))
example = x_train[idx: idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(6)


## Refactoring with core `torch.nn` components


### Using `torch.nn.functional` for stateless computation

In [ ]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [ ]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

accuracy comparison -  tensor([0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
        1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0.])
tensor(1.2205, grad_fn=<NllLossBackward0>) tensor(0.7500)


### Using torch.nn.Module to define functions whose state is given by `torch.nn